In [1]:
import os, json
QnA_dir_path = 'final_1000_0425_human_checked.json'
with open(QnA_dir_path, 'r') as f:
    file_data = json.load(f)

In [2]:
# len(file_data)

In [38]:
with open('/home/luhan/real_pubmedqa.json', 'r') as f:
    ques = json.load(f)

In [39]:
answer_list = [q['completion'] for q in ques]

In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = 'internlm2_20b_protein_lora_v3_pretrain_modified'
os.environ['HUGGINGFACE_HUB_CACHE'] = "/data/llm_models/huggingface/hub"

model_path = f'/data/llm_models/{model_name}'
if 'models--' in model_name:
    fs = f'/data/llm_models/huggingface/hub/{model_name}/snapshots/'
    model_path = fs + os.listdir(f'/data/llm_models/huggingface/hub/{model_name}/snapshots/')[0]
benchmark_size = 1000
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True, device_map = "auto").eval()

In [40]:
ques = ques[:500]
chat_model = ('chat' in model_name) or ('Chat' in model_name)
if 'Baichuan' in model_name:
    chat_model = False
inputs = []
tokenizer.pad_token = tokenizer.eos_token
if ('struct' or 'it') in model_name:
    for q in ques:
        q = q['prompt']
        a = tokenizer.apply_chat_template([{"role": "user", "content": q}], return_tensors="pt").to("cuda")
        inputs.append(a)
elif not chat_model:
    for q in ques:
        q = q['prompt'] + 'The corrct option is'
        a = tokenizer(q, return_tensors="pt", padding=True)
        input_ids = a.input_ids.to('cuda')
        inputs.append(input_ids)
else:
    inputs = [q['prompt'] + 'The corrct option is' for q in ques]

print(len(inputs))

In [41]:
from tqdm import tqdm
output_list = []

In [ ]:

temp = 0.1
mnt = 10
for q in tqdm(inputs[len(output_list):]):
    if chat_model:
        try:
            if 'Mistral' in model_name:
                output_list.append(model.chat(tokenizer, q, do_sample=True, max_new_tokens=mnt, temperature=temp, history=[], eos_token_id=2, pad_token_id=2))
            else:
                output_list.append(model.chat(tokenizer, q, max_new_tokens=mnt, do_sample=True, temperature=temp, history=[]))
        except:
            output_list.append(model.generate(q, max_new_tokens=mnt, do_sample=True, temperature=temp))
    else:
        if 'Mistral' in model_name:
            output_list.append(model.generate(q, max_new_tokens=mnt,do_sample=True, temperature=temp, eos_token_id=2, pad_token_id=2))
        elif 'llama3-8B' or 'Llama-3' in model_name:
            output_list.append(model.generate(q, max_new_tokens=mnt,do_sample=True, temperature=temp, eos_token_id=128001, pad_token_id=128001))
        elif 'falcon' in model_name:
            output_list.append(model.generate(q, max_new_tokens=mnt,do_sample=True, temperature=temp, eos_token_id=11, pad_token_id=11))
        else:
            output_list.append(model.generate(q, max_new_tokens=mnt, do_sample=True,temperature=temp))

In [31]:
question = [q['prompt'] for q in ques]

In [32]:
after = []
lst = [tokenizer.decode(i[0], skip_special_tokens=True) for i in output_list]
if not chat_model:
    for i, j in zip(lst, question):
        after.append(i.replace(j, ''))
else:
    for i, j in zip(output_list, question):
        after.append(i[0].replace(j, ''))


In [33]:
# other = True

In [34]:

# after = [a[15:]  for a in after]
import re

organized_output_list = []
for i in after:  # Assuming 'after' is a list of strings you're iterating through
    # if ('A) no' or 'B) yes') in i:
    #     organized_output_list.append('x')
    #     continue
    # eos = '<|im_end|>'
    # if eos in i:
    #     index = i.rfind(eos) + len(eos)
    #     i = i[index:].strip()
    # if other:
    #     for o in after:
    #         try:
    #             organized_output_list.append(re.search(r'\d+', o).group())
    #         except:
    #             organized_output_list.append("None")
    #     break
    if 'Answer' in i:
        index = i.rfind('Answer') + len('Answer')
        i = i[index:].strip()
    temp = re.search(r'[A-D]',
                     i)  # This regex matches the first uppercase letter
    if temp:
        organized_output_list.append(
            temp.group())  # Append the first uppercase letter found
    else:
        organized_output_list.append(
            'x')  # Append 'x' if no uppercase letter is found

In [37]:

num_correct = 0
for i in range(len(organized_output_list)):
    if organized_output_list[i] == answer_list[i]:
        num_correct += 1
num_correct / len(organized_output_list)

In [36]:
# inputs = []
# for q in question:
#     a = tokenizer.apply_chat_template([{"role": "user", "content": q}], return_tensors="pt").to("cuda")
#     inputs.append(a)